<a href="https://colab.research.google.com/github/Walt9819/factores-impacto-desempleo-mexico/blob/main/Python/FactoresDeImpactoEnDesempleoMX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipo 18

BEDU - Santander: Módulo 2. Procesamiento de Datos con Python

Proyecto del equipo 18

**Integrantes:**
- Irene López [ver perfil](https://www.linkedin.com/in/irene-lopez-rodriguez/)
- Laura Lozano [ver perfil](https://www.linkedin.com/in/laura-lozano-bautista/)
- Enrique Rodríguez [ver perfil](https://www.linkedin.com/in/enrique-rodriguez97/)
- Walter Rosales [ver perfil]()

# Factores de Impacto en el Desempleo y la Recuperación Económica en México

## Objetivo
Analizar la situación del empleo en México durante los tres primeros trimestres de 2020, asimismo, calcular la probabilidad de estar desempleado conforme a características sociodemográficas y así identificar los perfiles de los mexicanos más vulnerables ante el deterioro del mercado laboral derivado de la situación que vivimos hoy en día.

# Primera descarga de datos desde el INEGI

**NOTA:** solo descargar si no están los datos dispoibles en el directorio de trabajo o si se desean actualizar los datos para incluir más trimestres de diferentes años

In [1]:
# run just once
!pip3 install zipfile38
!pip3 install dbfread

In [2]:
import requests, sys, io # lectura desde URL
import zipfile38 as zipfile # apoyos para lecturas de ZIP

import pandas as pd # usar dataframes
import os # manipulación de archivos en el sistema

from dbfread import DBF # lectura de archivos .dbf

zips = ["SDEMT0420", "SDEMT0520", "SDEMT0620", "SDEMT120", "enoen_sdemt320", "SDEMT420"]

for zip in zips:
  r = requests.get(f"https://github.com/Walt9819/factores-impacto-desempleo-mexico/blob/main/R/Project/enoe_sdem/{zip}.zip?raw=true")
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall("")


# Columnas a leer
columnas = ["ENT", "MUN", "SEX", "EDA", "NIV_INS", "RAMA", "CLASE2", "PER"]

# Conjunto de datos a llenar
data = pd.DataFrame()

# Leer todos los archivos `.dbf`
for file in os.listdir():
    if file.endswith(".dbf"):
      table = DBF(file, load=True)
      print(f"Load {file} succesfully\nTransforming data..")
      table = pd.DataFrame(iter(table))
      print("Adding data to main dataframe...")
      data = data.append(table[columnas])

#Export to data
data.to_csv("data_enoe.csv", encoding="utf-8", index=False) # exportar a un CVS (de forma local)

Load SDEMT0420.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load SDEMT120.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load enoen_sdemt320.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load SDEMT0620.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load SDEMT0520.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load ENOEN_SDEMT420.dbf succesfully
Transforming data..
Adding data to main dataframe...


# Lectura y limpieza de datos

In [3]:
datos = pd.read_csv("https://github.com/Walt9819/factores-impacto-desempleo-mexico/blob/main/Python/data/data_enoe.csv?raw=true", encoding="utf-8") # leer datos

datos.dropna(inplace=True) # eliminar valores NA

# Se omiten los registros de las personas no economicamente activas y no disponibles 
# [clase2 == 4], a los menores de 15 años y mayores de 65 años [eda <= 15 & eda >= 65] 
# y aquellos sin nivel educativo registrado [niv_ins == 5]
datos[(datos["EDA"] >= 15) & (datos["EDA"] <= 65) & (datos["CLASE2"] <= 3) & (datos["CLASE2"] != 0) & (datos["NIV_INS"] <= 4)]

,ENT,MUN,SEX,EDA,NIV_INS,RAMA,CLASE2,PER
0,9,7.0,1.0,48.0,4,4,1,220
3,9,7.0,1.0,16.0,3,0,2,220
6,9,15.0,2.0,57.0,4,0,3,220
7,9,15.0,2.0,49.0,3,4,1,220
8,9,3.0,1.0,44.0,2,2,1,220
...,...,...,...,...,...,...,...,...
1164557,7,89.0,1.0,19.0,3,4,1,420
1164560,7,89.0,2.0,52.0,3,4,1,420
1164561,7,89.0,1.0,28.0,4,4,1,420
1164562,7,89.0,1.0,37.0,4,4,1,420
